In [ ]:
%pip install sklearn scikit-image h5py

In [6]:
# load data
import json
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tqdm

# map
with open("data02/database/database_lite.json","r") as f:
    m_idx = json.load(f)
    m_imgs = np.array(m_idx["im_paths"])
    m_loc=np.array(m_idx["loc"])

# query
with open("data02/query/query_lite.json","r") as f:
    q_idx=json.load(f)
    q_imgs=np.array(q_idx["im_paths"])
    q_loc=np.array(q_idx["loc"])


In [51]:
# Create descriptors
import os
import skimage
from skimage.feature import ORB
from skimage.color import rgb2gray

# Initialize the ORB descriptor
descriptor_extractor = ORB(n_keypoints=10000)
# Initialize the data structure that will contain all the descriptors
descriptors = []
responses = []

# Loop over map images
for img_name in m_imgs:
    img = plt.imread(os.path.join('data02/', img_name))
    img = rgb2gray(img)
    
    # Extract ORB descriptors
    descriptor_extractor.detect_and_extract(img)  
    # keypoints1 = descriptor_extractor.keypoints  # position of the points (not interesting for us)
    descriptors_img = descriptor_extractor.descriptors  # descriptors (the feature vectors)
    responses_img = descriptor_extractor.responses
    # Accumulate the computed descriptors
    descriptors.append(descriptors_img)
    responses.append(responses_img)

In [53]:
# Ns for which a subset is created
ns = [50, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]
descriptors_ns = [] # will contain subsets of the descriptors for each N in Ns
for n in ns:
    descriptors_n = []
    for index in range(len(descriptors)):
        best_indices = responses[index].argsort()[::-1][:n]
        descriptors_n.extend(descriptors[index][best_indices])
    descriptors_ns.append(descriptors_n)


In [1]:
import pickle
# save descriptors (uncomment if you want to save the computed descriptors)
#f = open('./ORB-descriptors-original.bin', 'wb')
#data = pickle.dump(descriptors, f)
#f.close()
#f = open('./ORB-descriptors-for-ns.bin', 'wb')
#data = pickle.dump(descriptors_ns, f)
#f.close()

# load pre-computed descriptors
f = open('./ORB-descriptors-original.bin', 'rb')
descriptors = pickle.load(f)
f.close()
f = open('./ORB-descriptors-for-ns.bin', 'rb')
descriptors_ns = pickle.load(f)
f.close()


In [15]:
import sklearn
from sklearn.cluster import KMeans

# clustering
K = 32  # number of clusters (equivalent to the number of words) we want to estimate
num_initialization = 5 # Number of time the k-means algorithm will be run with different centroid seeds.

# Run the k-means clustering
kmeans = KMeans(n_clusters=K, random_state=0, n_init=num_initialization, verbose=1)
clusters = kmeans.fit(descriptors_ns[9])  # we use the descriptors extracted from the map (training) images before
centroids = clusters.cluster_centers_

print("Shape of the centroids matrix: ", centroids.shape)
print("We computed ", centroids.shape[0], "centroids of lengh ", centroids.shape[1], " (the same of the descriptor)")
# Rememeber: the centroids can be considered as the words that compose our documents 
# -> in this case the basic components of the images

Initialization complete
Iteration 0, inertia 163748949.0.
Iteration 1, inertia 96636313.29958874.
Iteration 2, inertia 95698901.235313.
Iteration 3, inertia 95382570.9563854.
Iteration 4, inertia 95216972.51252605.
Iteration 5, inertia 95110546.84568006.
Iteration 6, inertia 95033326.96594462.
Iteration 7, inertia 94974698.80934578.
Iteration 8, inertia 94930214.00860327.
Iteration 9, inertia 94896306.93101843.
Iteration 10, inertia 94870667.01423198.
Iteration 11, inertia 94850959.12189871.
Iteration 12, inertia 94835230.76458235.
Iteration 13, inertia 94822023.50296646.
Iteration 14, inertia 94810534.53777608.
Iteration 15, inertia 94800242.81642118.
Iteration 16, inertia 94790756.10576613.
Iteration 17, inertia 94781857.7352466.
Iteration 18, inertia 94773452.35631515.
Iteration 19, inertia 94765653.81427526.
Iteration 20, inertia 94758291.24114628.
Iteration 21, inertia 94751409.22913578.
Iteration 22, inertia 94745012.77149835.
Iteration 23, inertia 94739071.79984233.
Iteration 24

In [16]:
f = open('./knn_centroids_9.bin', 'wb')
data = pickle.dump(centroids, f)
f.close()

# load pre-computed descriptors
f = open('./knn_centroids_9.bin', 'rb')
centroids = pickle.load(f)
f.close()